In [5]:
from skimage import color
import cv2
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import pandas as pd

csv_path = "../../data/train.csv"
img_dir = "../../data/train_ims"
output_dir = "../../data/train_sift_ims"
data = pd.read_csv(csv_path)

X = []
y = []

# 创建 SIFT 对象
sift = cv2.SIFT_create()

for _, row in tqdm(data.iterrows(), total=len(data), desc="Processing Images"):
    img_path = os.path.join(img_dir, row.iloc[0])
    label = int(row.iloc[1])
    img_name = os.path.splitext(row.iloc[0])[0]
    img = Image.open(img_path).convert("RGB")
    
    # 转换为灰度图像
    img_gray = color.rgb2gray(np.array(img))
    img_gray = (img_gray * 255).astype(np.uint8)  # 转换为 8 位灰度图像
    img_gray = cv2.resize(img_gray, (128, 128), interpolation=cv2.INTER_CUBIC)
    # 检测 SIFT 特征点并计算描述符
    keypoints, descriptors = sift.detectAndCompute(img_gray, None)
    
    if descriptors is not None:
        # 保存 SIFT 特征点可视化图像
        img_sift = cv2.drawKeypoints(img_gray, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    else:
        img_sift = img_gray
    
    sift_image_path = os.path.join(output_dir, f"{img_name}.jpg")
    cv2.imwrite(sift_image_path, img_sift)

        # 将描述符展平并添加到特征列表
        # X.append(descriptors.flatten())
        # y.append(label)

# 将特征和标签转换为 NumPy 数组
X = np.array(X, dtype=object)  # 使用 dtype=object 以支持不同长度的描述符
y = np.array(y)

Processing Images:   0%|          | 0/50000 [00:00<?, ?it/s]

Processing Images: 100%|██████████| 50000/50000 [01:20<00:00, 622.59it/s]
